In [2]:
import sqlite3
import duckdb
import pandas as pd

from tqdm import tqdm

In [22]:
# Connect to the SQLite database
sqlite_conn = sqlite3.connect("./data/danbooru2023.db")
duckdb_conn = duckdb.connect(f"./data/danbooru2023.duckdb")

In [24]:
# Get list of all tables
cursor = sqlite_conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
dfs = []

for table in tqdm(tables):
    table_name = table[0]
    # Read the SQLite table into a pandas DataFrame
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", sqlite_conn)
    df.to_parquet(f"./data/{table_name}.parquet")
    dfs.append(df)
    duckdb_conn.execute(f"CREATE TABLE {table_name} AS SELECT * FROM './data/{table_name}.parquet'")

  0%|          | 0/4 [00:00<?, ?it/s]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

 50%|█████     | 2/4 [01:28<01:13, 36.52s/it]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

100%|██████████| 4/4 [06:56<00:00, 104.21s/it]


In [25]:
duckdb_conn.close()
sqlite_conn.close()

In [26]:
import shutil
shutil.copyfile("./data/danbooru2023.duckdb", "./data/danbooru2023-no-index.duckdb")
duckdb_conn = duckdb.connect(f"./data/danbooru2023.duckdb")
duckdb_conn.execute("CREATE INDEX post_id_relation ON posttagrelation(post_id)")
duckdb_conn.execute("CREATE INDEX tag_id_relation ON posttagrelation(tag_id)")
duckdb_conn.execute("CREATE UNIQUE INDEX post_id ON post(id)")
duckdb_conn.execute("CREATE UNIQUE INDEX tag_name ON tag(name)")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [3]:
import shutil
# shutil.copyfile("./data/danbooru2023.db", "./data/danbooru2023-no-index.db")
conn = sqlite3.connect("./data/danbooru2023-no-index.db")
drop_statements = conn.execute(
    """
SELECT 'DROP INDEX IF EXISTS ' || name || ';'
FROM sqlite_master
WHERE type = 'index' AND name NOT LIKE 'sqlite_%';
""".strip()
)

# Execute each DROP INDEX statement
for statement in drop_statements:
    conn.execute(statement[0])

OperationalError: database table is locked

In [29]:
conn.close()